In [1]:
from models import * 
from Functions import * 
import h5py
import torch.optim as optim
import torch.nn as nn
from torch.optim import lr_scheduler

Using TensorFlow backend.


In [2]:
#Working on GPU (Preferably)
device =cuda()

GPU is available


In [5]:
"""this can be used to download the files from Kaggle to google colab for example 
    Kaggle API key needs to be uploaded  """

from google.colab import files
upload=files.upload()
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!kaggle competitions download -c dreem-sleep-stages-2020 -p /content
#We need to decompress the files
!unzip \X_train.h5.zip

Saving kaggle.json to kaggle.json
 99% 1.00G/1.01G [00:10<00:00, 99.2MB/s]
100% 1.01G/1.01G [00:10<00:00, 103MB/s] 
100% 0.99G/1.00G [00:17<00:00, 71.3MB/s]
100% 1.00G/1.00G [00:17<00:00, 62.6MB/s]
  0% 0.00/195k [00:00<?, ?B/s]
100% 195k/195k [00:00<00:00, 199MB/s]
  0% 0.00/182k [00:00<?, ?B/s]
100% 182k/182k [00:00<00:00, 179MB/s]
Archive:  X_train.h5.zip
  inflating: X_train.h5              


#  EEG Data Feature Extraction


In [0]:
f= h5py.File('X_train.h5', 'r')
L=list(f.keys())
y=pd.read_csv('y_train.csv')
y=y['sleep_stage'].values
df_eeg,n_measures_eeg,n_samples=Create_dataset(L[:7],f)
F_s=50 #Sampling Frequency
shape_eeg=(n_samples,n_measures_eeg,F_s*30)
X_train,y_train,X_val,y_val=Oversample(df_eeg,y,shape_eeg,split=.2,shuffle=False)

In [0]:
batch_size=64
train_loader=create_loader(X_train,y_train,batch_size)
val_loader=create_loader(X_val,y_val,batch_size)
classifier_eeg = Feature_extractor(n_measures_eeg,F_s)
classifier_eeg.to(device)
optimizer = optim.Adam(classifier_eeg.parameters(),lr=0.1)#,weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()
scheduler=lr_scheduler.StepLR(optimizer,step_size=10,gamma=0.1)

In [57]:
#Training EEG data Feature EXtractor 
epochs=50
Losses_v=[]
Losses_t=[]
for epoch in range(epochs):
  print('epoch: ',epoch)
  Losses_t+=Train_model(classifier_eeg,criterion,optimizer,train_loader,scheduler,device,mode='train')
  Losses_v+=Train_model(classifier_eeg,criterion,optimizer,val_loader,scheduler,device,mode='val')
#Saving State
w_eeg=classifier_eeg.state_dict()

epoch:  0
train Loss 0.6609  Acc : 75.11%  f1 score 0.75
val Loss 0.8080  Acc : 69.95%  f1 score 0.70
epoch:  1
train Loss 0.6244  Acc : 76.20%  f1 score 0.76
val Loss 0.7993  Acc : 70.21%  f1 score 0.71
epoch:  2
train Loss 0.6186  Acc : 76.61%  f1 score 0.77
val Loss 0.8013  Acc : 70.05%  f1 score 0.70
epoch:  3
train Loss 0.6185  Acc : 76.56%  f1 score 0.77
val Loss 0.8027  Acc : 69.89%  f1 score 0.70
epoch:  4
train Loss 0.6201  Acc : 76.55%  f1 score 0.77
val Loss 1.0115  Acc : 70.23%  f1 score 0.71


# Accel+Pulsometer Data Feature Extraction


In [0]:
y_pul=pd.read_csv('y_train.csv')
y_pul=y_pul['sleep_stage'].values
df_pul,n_measures_pul,n_samples=Create_dataset(L[-4:],f)
F_s_pul=10 #Sampling Frequency
shape_pul=(n_samples,n_measures_pul,F_s_pul*30)
X_train_pul,y_train_pul,X_val_pul,y_val_pul=Oversample(df_pul,y_pul,shape_pul,split=.2,shuffle=False)

In [0]:
batch_size=64
train_loader_pul=create_loader(X_train_pul,y_train_pul,batch_size)
val_loader_pul=create_loader(X_val_pul,y_val_pul,batch_size)
classifier_pul = Feature_extractor(n_measures_pul,F_s_pul)
classifier_pul.to(device)
optimizer = optim.Adam(classifier_pul.parameters(),lr=0.1)#,weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()
scheduler=lr_scheduler.StepLR(optimizer,step_size=10,gamma=0.1)

In [62]:
#Training Acceloremeter+Pulsometer Feature EXtractor 
epochs=5
Losses_v=[]
Losses_t=[]
for epoch in range(epochs):
  print('epoch: ',epoch)
  Losses_t+=Train_model(classifier_pul,criterion,optimizer,train_loader_pul,scheduler,device,mode='train')
  Losses_v+=Train_model(classifier_pul,criterion,optimizer,val_loader_pul,scheduler,device,mode='val')
#Saving State
w_pul=classifier_pul.state_dict()

epoch:  0
train Loss 1.0660  Acc : 58.95%  f1 score 0.59
val Loss 1.1392  Acc : 54.09%  f1 score 0.54
epoch:  1
train Loss 1.0601  Acc : 59.36%  f1 score 0.59
val Loss 1.1430  Acc : 54.19%  f1 score 0.54
epoch:  2
train Loss 1.0574  Acc : 59.08%  f1 score 0.59
val Loss 1.1514  Acc : 53.50%  f1 score 0.54
epoch:  3
train Loss 1.0607  Acc : 59.33%  f1 score 0.59
val Loss 1.1403  Acc : 53.16%  f1 score 0.53
epoch:  4
train Loss 1.0525  Acc : 59.24%  f1 score 0.59
val Loss 1.1254  Acc : 54.96%  f1 score 0.55


# LSTM Sequential   training


In [41]:
#Creating Sequential Datasets Here
lookback=3
dataX,dataY=Create_Sequential_data(df_eeg,lookback=lookback,n_measures=n_measures_eeg,labels=y)
dataX_pul,dataY_pul=Create_Sequential_data(df_pul,lookback=lookback,n_measures=n_measures_pul,labels=y_pul)
data=np.concatenate((dataX.reshape(dataX.shape[0:2]+(-1,)),dataX_pul.reshape(dataX_pul.shape[0:2]+(-1,))),axis=2)
"""We can use this To free Up memory if needed"""
del dataX,dataX_pul
"""------------------"""
dataX_train,dataX_val,dataY_train,dataY_val=train_test_split(data,dataY,test_size=.2,shuffle=False)
Seq_train_loader=create_loader(dataX_train,dataY_train,batch_size=batch_size)
Seq_val_loader=create_loader(dataX_val,dataY_val,batch_size=batch_size)
"""Same We can use this To free Up memory if needed"""
del dataX_train,dataX_val,data
"""------------------"""

'------------------'

In [0]:
seq=Seq_learn(classifier_eeg.n_features+classifier_pul.n_features,lookback=lookback,drop=0)
seq.to(device)
criterion = nn.CrossEntropyLoss()

### First we Freeze the feature Extractors wheights

In [0]:
classifier_eeg.load_state_dict(w_eeg)
classifier_pul.load_state_dict(w_pul)
for param in classifier_eeg.parameters():
    param.requires_grad = False
for param in classifier_pul.parameters():
    param.requires_grad = False
optimizer=optim.Adam([{'params':classifier_eeg.parameters(),'lr':0.001},
            {'params':classifier_pul.parameters(),'lr':0.001},
            {'params':seq.parameters()}],lr=0.1)#,weight_decay=1e-4)
scheduler=lr_scheduler.StepLR(optimizer,step_size=10,gamma=0.1)

In [70]:
epochs=10
Losses_v=[]
Losses_t=[]
models=(classifier_eeg,classifier_pul,seq)
for epoch in range(epochs):
  print('epoch: ',epoch)
  Losses_t+=Train_seq_model(models,criterion,optimizer,Seq_train_loader,scheduler,shape_eeg,shape_pul,device,lookback,mode='train')
  Losses_v+=Train_seq_model(models,criterion,optimizer,Seq_val_loader,scheduler,shape_eeg,shape_pul,device,lookback,mode='val')

epoch:  0
train Loss 0.8941  Acc : 78.97%  f1 score 0.79
val Loss 0.7523  Acc : 82.09%  f1 score 0.82
epoch:  1
train Loss 0.8922  Acc : 78.87%  f1 score 0.79
val Loss 0.8386  Acc : 82.35%  f1 score 0.82
epoch:  2
train Loss 0.9128  Acc : 79.09%  f1 score 0.79
val Loss 185.5197  Acc : 82.23%  f1 score 0.82
epoch:  3
train Loss 0.9282  Acc : 78.48%  f1 score 0.78
val Loss 3.7416  Acc : 82.26%  f1 score 0.82
epoch:  4
train Loss 0.8739  Acc : 79.41%  f1 score 0.79
val Loss 1.9615  Acc : 82.57%  f1 score 0.83
epoch:  5
train Loss 0.9085  Acc : 78.72%  f1 score 0.79
val Loss 0.7314  Acc : 82.50%  f1 score 0.83
epoch:  6
train Loss 0.9117  Acc : 78.92%  f1 score 0.79
val Loss 0.7362  Acc : 82.51%  f1 score 0.83
epoch:  7
train Loss 0.9109  Acc : 78.33%  f1 score 0.78
val Loss 0.7435  Acc : 82.20%  f1 score 0.82
epoch:  8
train Loss 0.8947  Acc : 79.30%  f1 score 0.79
val Loss 0.7454  Acc : 82.23%  f1 score 0.82
epoch:  9
train Loss 0.8973  Acc : 78.79%  f1 score 0.79
val Loss 0.7373  Acc : 

### Fine Tuning parameters along with the feature Extractors wheights

In [0]:
classifier_eeg.load_state_dict(w_eeg)
classifier_pul.load_state_dict(w_pul)
for param in classifier_eeg.parameters():
    param.requires_grad = True
for param in classifier_pul.parameters():
    param.requires_grad = True
optimizer=optim.Adam([{'params':classifier_eeg.parameters(),'lr':0.001},
            {'params':classifier_pul.parameters(),'lr':0.001},
            {'params':seq.parameters()}],lr=0.005)#,weight_decay=1e-4)
scheduler=lr_scheduler.StepLR(optimizer,step_size=10,gamma=0.1)

In [75]:
epochs=10
Losses_v=[]
Losses_t=[]
models=(classifier_eeg,classifier_pul,seq)
for epoch in range(epochs):
  print('epoch: ',epoch)
  Losses_t+=Train_seq_model(models,criterion,optimizer,Seq_train_loader,scheduler,shape_eeg,shape_pul,device,lookback,mode='train')
  Losses_v+=Train_seq_model(models,criterion,optimizer,Seq_val_loader,scheduler,shape_eeg,shape_pul,device,lookback,mode='val')

epoch:  0
train Loss 0.5000  Acc : 81.68%  f1 score 0.81
val Loss 0.4619  Acc : 83.36%  f1 score 0.83
epoch:  1
train Loss 0.5008  Acc : 81.36%  f1 score 0.81
val Loss 0.4597  Acc : 83.49%  f1 score 0.84
epoch:  2
train Loss 0.4987  Acc : 81.40%  f1 score 0.81
val Loss 6.9405  Acc : 83.28%  f1 score 0.83
epoch:  3
train Loss 0.5010  Acc : 81.15%  f1 score 0.81
val Loss 1.1146  Acc : 83.39%  f1 score 0.83
epoch:  4
train Loss 0.5029  Acc : 81.49%  f1 score 0.81
val Loss 7.1424  Acc : 83.50%  f1 score 0.84
epoch:  5
train Loss 0.4924  Acc : 81.68%  f1 score 0.82
val Loss 2.0651  Acc : 83.39%  f1 score 0.83
epoch:  6
train Loss 0.4955  Acc : 81.69%  f1 score 0.82
val Loss 1.7013  Acc : 83.76%  f1 score 0.84
epoch:  7
train Loss 0.4902  Acc : 81.72%  f1 score 0.82
val Loss 0.6659  Acc : 83.38%  f1 score 0.83
epoch:  8
train Loss 0.4958  Acc : 81.85%  f1 score 0.82
val Loss 0.4598  Acc : 83.50%  f1 score 0.84
epoch:  9
train Loss 0.5085  Acc : 81.34%  f1 score 0.81
val Loss 0.6868  Acc : 83

### Generating Testing Files For Kaggle 

In [0]:
"""If files were downloaded without Unzipping Uncomment"""
#!unzip \X_test.h5.zip

'If files were downloaded without Unzipping Uncomment'

In [0]:
f= h5py.File('X_test.h5', 'r')  
Preds=Test_seq_model((classifier_eeg,classifier_pul,seq),batch_size,f ,L[:7],L[-4:],F_s,F_s_pul,lookback,w_eeg,w_pul,device )
Preds.to_csv('submission.csv',index=True,header=True)